<a href="https://colab.research.google.com/github/tara3165/NLP-/blob/CompliantAnalysis/ComplaintAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
/Colab Notebooks/Complaint

drive  sample_data


In [10]:
cd drive

/content/drive


In [11]:
import pandas as pd
df_complaints=pd.read_csv('../drive/MyDrive/Colab Notebooks/Complaint/complaints.csv',low_memory=False)

In [12]:
df_complaints=df_complaints[~df_complaints['Consumer complaint narrative'].isna()]
Description_to_process='Consumer complaint narrative'

In [ ]:
#import spacy
#en = spacy.load("en_core_web_lg")
#stopwords = en.Defaults.stop_words


#spacy_stopwords=
domain_stopwords=['call', 'upon', 'still', 'nevertheless', 'down', 'every', 'forty', '‘re', 'always', 'whole', 'side', "n't", 'now', 'however', 'an', 'show', 'least', 'give', 'below', 'did', 'sometimes', 'which', "'s", 'nowhere', 'per', 'hereupon', 'yours', 'she', 'moreover', 'eight', 'somewhere', 'within', 'whereby', 'few', 'has', 'so', 'have', 'for', 'noone', 'top', 'were', 'those', 'thence', 'eleven', 'after', 'no', '’ll', 'others', 'ourselves', 'themselves', 'though', 'that', 'nor', 'just', '’s', 'before', 'had', 'toward', 'another', 'should', 'herself', 'and', 'these', 'such', 'elsewhere', 'further', 'next', 'indeed', 'bottom', 'anyone', 'his', 'each', 'then', 'both', 'became', 'third', 'whom', '‘ve', 'mine', 'take', 'many', 'anywhere', 'to', 'well', 'thereafter', 'besides', 'almost', 'front', 'fifteen', 'towards', 'none', 'be', 'herein', 'two', 'using', 'whatever', 'please', 'perhaps', 'full', 'ca', 'we', 'latterly', 'here', 'therefore', 'us', 'how', 'was', 'made', 'the', 'or', 'may', '’re', 'namely', "'ve", 'anyway', 'amongst', 'used', 'ever', 'of', 'there', 'than', 'why', 'really', 'whither', 'in', 'only', 'wherein', 'last', 'under', 'own', 'therein', 'go', 'seems', '‘m', 'wherever', 'either', 'someone', 'up', 'doing', 'on', 'rather', 'ours', 'again', 'same', 'over', '‘s', 'latter', 'during', 'done', "'re", 'put', "'m", 'much', 'neither', 'among', 'seemed', 'into', 'once', 'my', 'otherwise', 'part', 'everywhere', 'never', 'myself', 'must', 'will', 'am', 'can', 'else', 'although', 'as', 'beyond', 'are', 'too', 'becomes', 'does', 'a', 'everyone', 'but', 'some', 'regarding', '‘ll', 'against', 'throughout', 'yourselves', 'him', "'d", 'it', 'himself', 'whether', 'move', '’m', 'hereafter', 're', 'while', 'whoever', 'your', 'first', 'amount', 'twelve', 'serious', 'other', 'any', 'off', 'seeming', 'four', 'itself', 'nothing', 'beforehand', 'make', 'out', 'very', 'already', 'various', 'until', 'hers', 'they', 'not', 'them', 'where', 'would', 'since', 'everything', 'at', 'together', 'yet', 'more', 'six', 'back', 'with', 'thereupon', 'becoming', 'around', 'due', 'keep', 'somehow', 'n‘t', 'across', 'all', 'when', 'i', 'empty', 'nine', 'five', 'get', 'see', 'been', 'name', 'between', 'hence', 'ten', 'several', 'from', 'whereupon', 'through', 'hereby', "'ll", 'alone', 'something', 'formerly', 'without', 'above', 'onto', 'except', 'enough', 'become', 'behind', '’d', 'its', 'most', 'n’t', 'might', 'whereas', 'anything', 'if', 'her', 'via', 'fifty', 'is', 'thereby', 'twenty', 'often', 'whereafter', 'their', 'also', 'anyhow', 'cannot', 'our', 'could', 'because', 'who', 'beside', 'by', 'whence', 'being', 'meanwhile', 'this', 'afterwards', 'whenever', 'mostly', 'what', 'one', 'nobody', 'seem', 'less', 'do', '‘d', 'say', 'thus', 'unless', 'along', 'yourself', 'former', 'thru', 'he', 'hundred', 'three', 'sixty', 'me', 'sometime', 'whose', 'you', 'quite', '’ve', 'about', 'even']

#nltk_stopwords=
domain_stopwords.extend(['ve', 'am', 'can', 'shouldn', 'down', 'from', "wouldn't", 'as', 'be', 'are', 'too', 'through', "don't", 'does', 'a', 'but', 'now', 'some', 'an', "couldn't", 'we', 'below', 'against', 'here', 'won', 'did', 'yourselves', 'was', 'how', 'above', 'him', 'shan', 'it', 'which', "weren't", 'himself', 'its', 'most', 'the', 'wouldn', 'needn', 're', 'or', 'while', "mustn't", 'your', 'if', "hasn't", "shouldn't", 'yours', 'she', 'her', 'wasn', 'other', 'aren', 'any', 'll', 'off', 'couldn', 'ain', "wasn't", 'few', 'is', 'of', 'there', "isn't", 'than', "shan't", 'why', 'has', 'so', 'in', "didn't", 'only', 'have', 'itself', 'for', 'under', 'own', 'hasn', 'were', "doesn't", "needn't", 'those', 'isn', 'out', 'very', "won't", 'until', 'hers', 'after', "it's", 'up', 'they', 'their', 'not', 'doing', 'no', 'them', 'where', 'ourselves', 'themselves', "aren't", 'our', 'on', 'that', 'theirs', 'nor', 't', 'ours', 'at', 's', 'again', 'same', 'over', 'just', 'ma', 'because', 'who', 'mightn', "you've", 'before', 'by', 'more', "should've", 'being', 'had', 'weren', 'this', "mightn't", 'm', 'with', 'should', "haven't", "hadn't", 'what', "that'll", "you're", 'during', 'haven', 'herself', 'and', 'these', 'such', 'further', 'mustn', 'do', "you'd", 'having', 'didn', 'd', 'hadn', 'y', 'yourself', 'his', 'into', 'once', 'each', 'don', 'all', 'then', 'both', 'when', 'he', 'will', 'me', 'whom', 'i', 'my', 'you', 'to', 'between', 'myself', 'doesn', 'about', 'been', 'o', "you'll", "she's"])
domain_stopwords.extend(['hasnt', 'thick', 'interest', 'eg', 'computer', 'couldnt', 'didn', 'co', 'system', 'bill', 'kg', 'mill', 'un', 'don', 'thin', 'detail', 'de', 'found', 'cry', 'ltd', 'find', 'etc', 'inc', 'fire', 'con', 'cant', 'doesn', 'fill', 'amoungst', 'ie', 'describe', 'km', 'sincere',
"'d", "'d", "'s", "'re", "n't", "'d", "'m", "'m", "'m", "n't", "'re", "'ll", "'re", "'ll", "'ve", "n't", "'s", "'s", "'ve", "'ll", 'ca', "'ve"])

domain_stopwords.extend(["gimme", "lemme", "cause", "'cuz", "imma", "gonna", "wanna",
                   "gotta", "hafta", "woulda", "coulda", "shoulda", "howdy","day"])

#add product/subproduct to domain_stop word so we can  findout the underlying issues
domain_stopwords.extend(df_complaints['Sub-product'].unique().tolist())
domain_stopwords.extend(df_complaints['Product'].unique().tolist())
domain_stopwords.extend(["chase","credit card", "company","day","bank","hour","express",'credit','card','SYNCHRONY FINANCIAL','synchrony','financial'])
domain_stopwords.extend(["Ocwen Financial Corporation", "Ocwen","corporation","financial corporation","Paypal Holdings, Inc","paypal","holdings","paypal holdings"])

#add any other frequent word to domain_stopword

#lower case
domain_stopwords=[str(w).lower() for w in list(domain_stopwords)]


In [13]:
df_complaints['Year']=pd.to_datetime(df_complaints['Date received']) .dt.year
df_complaints['Month']=pd.to_datetime(df_complaints['Date received']) .dt.month
df_complaints['Day']=pd.to_datetime(df_complaints['Date received']) .dt.day

In [ ]:
set(df_complaints['Year'])

{2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024}

In [3]:
!pip install BERTopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 17.8 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.7 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039290 sha256=f6fe48b6a636c086d15754245bfee6449a86ca11f05c56e3de9e4beff047d7ce
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for umap-

In [ ]:
from bertopic import BERTopic
data=df_complaints[df_complaints['Year']==2023]
data=data[data['Month'].isin([1])]
data=data[data['Day'].isin([1,2])]#,3,4,5,6
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(data[Description_to_process])



2024-03-12 10:03:04,200 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/49 [00:00<?, ?it/s]

2024-03-12 10:03:20,853 - BERTopic - Embedding - Completed ✓
2024-03-12 10:03:20,854 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-12 10:03:33,835 - BERTopic - Dimensionality - Completed ✓
2024-03-12 10:03:33,837 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-12 10:03:33,898 - BERTopic - Cluster - Completed ✓
2024-03-12 10:03:33,906 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-12 10:03:34,247 - BERTopic - Representation - Completed ✓


In [ ]:
!wget https://huggingface.co/TheBloke/OpenHermes-2.5-Mistral-7B-GGUF/resolve/main/openhermes-2.5-mistral-7b.Q4_K_M.gguf
# !wget https://huggingface.co/TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/resolve/main/dolphin-2.7-mixtral-8x7b.Q3_K_M.gguf

--2024-03-12 10:07:24--  https://huggingface.co/TheBloke/OpenHermes-2.5-Mistral-7B-GGUF/resolve/main/openhermes-2.5-mistral-7b.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.102, 13.33.33.55, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/65/3b/653b1cc7a4558cd22c0ac02df471105878a080cb5694622ccd566697c574592f/cd58120326971c71c0590f6b7084a0744e287ce9c67275d8b4bf34a5947d950b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27openhermes-2.5-mistral-7b.Q4_K_M.gguf%3B+filename%3D%22openhermes-2.5-mistral-7b.Q4_K_M.gguf%22%3B&Expires=1710497245&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMDQ5NzI0NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzY1LzNiLzY1M2IxY2M3YTQ1NThjZDIyYzBhYzAyZGY0NzExMDU4NzhhMDgwY2I1Njk0NjIyY2NkNTY2Njk3YzU3NDU5MmYv

In [4]:
pip install keyphrase-vectorizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 28.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [26]:
from keyphrase_vectorizers import KeyphraseCountVectorizer

docs = ["""Supervised learning is the machine learning task of learning a function that
         maps an input to an output based on example input-output pairs. It infers a
         function from labeled training data consisting of a set of training examples.
         In supervised learning, each example is a pair consisting of an input object
         (typically a vector) and a desired output value (also called the supervisory signal).
         A supervised learning algorithm analyzes the training data and produces an inferred function,
         which can be used for mapping new examples. An optimal scenario will allow for the
         algorithm to correctly determine the class labels for unseen instances. This requires
         the learning algorithm to generalize from the training data to unseen situations in a
         'reasonable' way (see inductive bias).""",

        """Keywords are defined as phrases that capture the main topics discussed in a document.
        As they offer a brief yet precise summary of document content, they can be utilized for various applications.
        In an information retrieval environment, they serve as an indication of document relevance for users, as the list
        of keywords can quickly help to determine whether a given document is relevant to their interest.
        As keywords reflect a document's main topics, they can be utilized to classify documents into groups
        by measuring the overlap between the keywords assigned to them. Keywords are also used proactively
        in information retrieval."""]

# Init default vectorizer.
vectorizer = KeyphraseCountVectorizer()

# Print parameters
print(vectorizer.get_params())

{'binary': False, 'custom_pos_tagger': None, 'dtype': <class 'numpy.int64'>, 'lowercase': True, 'max_df': None, 'min_df': None, 'pos_pattern': '<J.*>*<N.*>+', 'spacy_exclude': None, 'spacy_pipeline': 'en_core_web_sm', 'stop_words': 'english', 'workers': 1}


In [27]:
kw_model.extract_keywords(docs, vectorizer=vectorizer)

[[('supervised learning algorithm', 0.6992),
  ('supervised learning', 0.6779),
  ('training data', 0.5271),
  ('training examples', 0.4668),
  ('class labels', 0.389)],
 [('document relevance', 0.633),
  ('keywords', 0.6046),
  ('information retrieval', 0.5792),
  ('information retrieval environment', 0.5166),
  ('document content', 0.3988)]]

In [46]:

document_keyphrase_matrix = vectorizer.fit_transform(docs).toarray()
print(document_keyphrase_matrix)

[[3 0 3 1 1 0 1 1 0 3 1 0 0 1 1 0 1 1 0 3 0 1 2 1 3 1 3 0 1 0 0 0 0 1 0 0
  0 1 1 1 1 0]
 [0 2 0 0 0 1 0 0 1 0 0 1 5 0 0 1 0 0 1 0 1 0 0 0 0 0 0 2 0 1 5 1 1 0 1 1
  1 0 0 0 0 1]]


In [47]:
# After learning the keyphrases, they can be returned.
keyphrases = vectorizer.get_feature_names_out()
print(keyphrases)

['output' 'main topics' 'algorithm' 'output pairs' 'optimal scenario'
 'various applications' 'output value' 'way' 'overlap' 'input'
 'supervisory signal' 'document content' 'keywords' 'set' 'vector'
 'phrases' 'unseen situations' 'unseen instances' 'indication'
 'training data' 'groups' 'input object' 'example' 'new examples'
 'function' 'task' 'supervised learning' 'information retrieval'
 'supervised learning algorithm' 'interest' 'document'
 'information retrieval environment' 'precise summary' 'inductive bias'
 'users' 'document relevance' 'list' 'pair' 'training examples'
 'class labels' 'machine' 'documents']


In [6]:
pip install keybert

  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.8.4-py3-none-any.whl size=39199 sha256=ae791abc7b61513a42f1fed68471e113d54fdb88a82392f058f229725c44f5b2
  Stored in directory: /root/.cache/pip/wheels/97/ef/4c/6588bd7072b0cc04225b40e639b991e49ebd4e21fb81f0acee
Successfully built keybert


In [7]:
from keybert import KeyBERT
# Init KeyBERT
kw_model = KeyBERT()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from keyphrase_vectorizers import KeyphraseCountVectorizer

kw_model.extract_keywords(docs=docs, vectorizer=KeyphraseCountVectorizer())

2024-03-12 12:59:06,577 - KeyphraseVectorizer - INFO - It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.
INFO:KeyphraseVectorizer:It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[[('supervised learning algorithm', 0.6992),
  ('supervised learning', 0.6779),
  ('training data', 0.5271),
  ('training examples', 0.4668),
  ('class labels', 0.389)],
 [('document relevance', 0.633),
  ('keywords', 0.6046),
  ('information retrieval', 0.5792),
  ('information retrieval environment', 0.5166),
  ('document content', 0.3988)]]

In [16]:
from bertopic import BERTopic
data=df_complaints[df_complaints['Year']==2023]
data=data[data['Month'].isin([1])]
data=data[data['Day'].isin([1])]#,3,4,5,6
topic_model = BERTopic(verbose=True,vectorizer_model=KeyphraseCountVectorizer())
topics, probs = topic_model.fit_transform(data[Description_to_process])


2024-03-12 13:03:08,021 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

2024-03-12 13:04:26,048 - BERTopic - Embedding - Completed ✓
2024-03-12 13:04:26,056 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-12 13:04:37,646 - BERTopic - Dimensionality - Completed ✓
2024-03-12 13:04:37,647 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-12 13:04:37,700 - BERTopic - Cluster - Completed ✓
2024-03-12 13:04:37,712 - BERTopic - Representation - Extracting topics from clusters using representation models.
/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)
2024-03-12 13:04:59,983 - BERTopic - Representation - Completed ✓


In [19]:
topic_model.get_document_info(data[Description_to_process])

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,I deposited a bad check from a new job unbekno...,0,0_xxxx_xxxx xxxx_xxxx xxxx xxxx_xxxx xxxx xxxx...,"[xxxx, xxxx xxxx, xxxx xxxx xxxx, xxxx xxxx xx...",[Accounts have violated my Rights. 15 U.S.C 16...,xxxx - xxxx xxxx - xxxx xxxx xxxx - xxxx xxxx ...,1.000000,False
1,TransUnion under 15 USC 1681 is an ELABORATE M...,0,0_xxxx_xxxx xxxx_xxxx xxxx xxxx_xxxx xxxx xxxx...,"[xxxx, xxxx xxxx, xxxx xxxx xxxx, xxxx xxxx xx...",[Accounts have violated my Rights. 15 U.S.C 16...,xxxx - xxxx xxxx - xxxx xxxx xxxx - xxxx xxxx ...,1.000000,False
2,I got a XXXX XXXX XXXX XXXX from XXXX XXXX XXX...,0,0_xxxx_xxxx xxxx_xxxx xxxx xxxx_xxxx xxxx xxxx...,"[xxxx, xxxx xxxx, xxxx xxxx xxxx, xxxx xxxx xx...",[Accounts have violated my Rights. 15 U.S.C 16...,xxxx - xxxx xxxx - xxxx xxxx xxxx - xxxx xxxx ...,1.000000,False
3,In accordance with the fair credit reporting a...,1,1_section_states_usc_reporting,"[section, states, usc, reporting, fair credit ...",[In accordance with the fair credit reporting ...,section - states - usc - reporting - fair cred...,0.731175,True
4,I have been making payments on my loans since ...,0,0_xxxx_xxxx xxxx_xxxx xxxx xxxx_xxxx xxxx xxxx...,"[xxxx, xxxx xxxx, xxxx xxxx xxxx, xxxx xxxx xx...",[Accounts have violated my Rights. 15 U.S.C 16...,xxxx - xxxx xxxx - xxxx xxxx xxxx - xxxx xxxx ...,1.000000,False
...,...,...,...,...,...,...,...,...
462,A unit was leased from XX/XX/XXXX to XX/XX/XXX...,0,0_xxxx_xxxx xxxx_xxxx xxxx xxxx_xxxx xxxx xxxx...,"[xxxx, xxxx xxxx, xxxx xxxx xxxx, xxxx xxxx xx...",[Accounts have violated my Rights. 15 U.S.C 16...,xxxx - xxxx xxxx - xxxx xxxx xxxx - xxxx xxxx ...,1.000000,False
463,Hello the bank which is Hungtion national bank...,0,0_xxxx_xxxx xxxx_xxxx xxxx xxxx_xxxx xxxx xxxx...,"[xxxx, xxxx xxxx, xxxx xxxx xxxx, xxxx xxxx xx...",[Accounts have violated my Rights. 15 U.S.C 16...,xxxx - xxxx xxxx - xxxx xxxx xxxx - xxxx xxxx ...,1.000000,False
464,I am attempting to notify the bureau that the ...,0,0_xxxx_xxxx xxxx_xxxx xxxx xxxx_xxxx xxxx xxxx...,"[xxxx, xxxx xxxx, xxxx xxxx xxxx, xxxx xxxx xx...",[Accounts have violated my Rights. 15 U.S.C 16...,xxxx - xxxx xxxx - xxxx xxxx xxxx - xxxx xxxx ...,1.000000,False
465,I do annual background checks and found my nam...,0,0_xxxx_xxxx xxxx_xxxx xxxx xxxx_xxxx xxxx xxxx...,"[xxxx, xxxx xxxx, xxxx xxxx xxxx, xxxx xxxx xx...",[Accounts have violated my Rights. 15 U.S.C 16...,xxxx - xxxx xxxx - xxxx xxxx xxxx - xxxx xxxx ...,1.000000,False


In [22]:
len(topic_model.vectorizer_model.get_feature_names_out())

3154

In [32]:
topic_model.representation

AttributeError: 'BERTopic' object has no attribute 'representation'

In [29]:
for x in topic_model.vectorizer_model.get_feature_names_out():
  print(x)

bad faith
pictures
reporting agency
auto loan payment history
midland credit xxxx address
adjustment
xxxx bureaus
currency
additional late payment history
service xxxx xx
xxxx hrs total
perjury
process
credit profile
personal information
arbitrary categorization
business days
applicable federal consumer
illegal practices
il xxxx xxxx number xxxx
description
fraudulent usage
dozens
today date plusfour
website
actual damages
names
furniture
coin
organization
actual recipient
federal trade commission
total cash
fha regulation
american express card
attached letters
husband
midland credit xxxx
adequate notice
full accounting ledger
explicit reservation
trade lines
federal law
adverse actions
total sale price
appeal
excessive number
midland credit management
xxxx xxxx xxxx xx
self
accounts
multiple phone calls
nonpublic information
formal complaint
debt collectors
amex account
consumers information
schoolsfirst
past due bill
wishes
next few weeks
incorrect spellings
delivery
section 604a sec

In [28]:
kw_model.extract_keywords(docs=data[Description_to_process], vectorizer=topic_model.vectorizer_model)

KeyError: 386

In [ ]:
import json
import time
import itertools
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from typing import Mapping, Any, List, Tuple

try:
    from bertopic import BERTopic
except ImportError:
    pass

try:
    from contextualized_topic_models.models.ctm import CombinedTM
    from contextualized_topic_models.utils.data_preparation import (
        TopicModelDataPreparation,
    )
    import nltk

    nltk.download("stopwords")
    from nltk.corpus import stopwords
except ImportError:
    pass


import gensim
import gensim.corpora as corpora
from gensim.models import ldaseqmodel


class Trainer:
    """Train and evaluate a topic model

    Arguments:
        dataset: The dataset to be used, should be a string and either a
                 dataset found in OCTIS or a custom dataset
        model_name: The name of the topic model to be used:
                        * BERTopic

        params: The parameters of the model to be used
        topk: The top n words in each topic to include
        custom_dataset: Whether a custom dataset is used
        bt_embeddings: Pre-trained embeddings used in BERTopic to speed
                       up training.
        bt_timestamps: Timestamps used in BERTopic for dynamic
                       topic modeling
        bt_nr_bins: Number of bins to create from timestamps in BERTopic
        custom_model: A custom BERTopic
        verbose: Control the verbosity of the trainer

    Usage:

    ```python
    from evaluation import Trainer
    dataset, custom = "20NewsGroup", False
    params = {"num_topics": [(i+1)*10 for i in range(5)], "random_state": 42}

    trainer = Trainer(dataset=dataset,
                      model_name="LDA",
                      params=params,
                      custom_dataset=custom,
                      verbose=True)
    ```

    This trainer focused on iterating over all combinations of parameters in `params`.
    In the example above, we iterate over different number of topics.
    """

    def __init__(
        self,
        dataset: str,
        model_name: str,
        params: Mapping[str, Any],
        topk: int = 10,
        custom_dataset: bool = False,
        bt_embeddings: np.ndarray = None,
        bt_timestamps: List[str] = None,
        bt_nr_bins: int = None,
        custom_model=None,
        verbose: bool = True,
    ):
        self.dataset = dataset
        self.custom_dataset = custom_dataset
        self.model_name = model_name
        self.params = params
        self.topk = topk
        self.timestamps = bt_timestamps
        self.nr_bins = bt_nr_bins
        self.embeddings = bt_embeddings
        self.ctm_preprocessed_docs = None
        self.custom_model = custom_model
        self.verbose = verbose

        # Prepare data and metrics
        self.data = self.get_dataset()
        self.metrics = self.get_metrics()

        # CTM
        self.qt_ctm = None
        self.training_dataset_ctm = None

    def train(self, save: str = False) -> Mapping[str, Any]:
        """Train a topic model

        Arguments:
            save: The name of the file to save it to.
                  It will be saved as a .json in the current
                  working directory

        Usage:

        ```python
        from evaluation import Trainer
        dataset, custom = "20NewsGroup", False
        params = {"num_topics": [(i+1)*10 for i in range(5)], "random_state": 42}

        trainer = Trainer(dataset=dataset,
                        model_name="LDA",
                        params=params,
                        custom_dataset=custom,
                        verbose=True)
        results = trainer.train(save="LDA_results")
        ```
        """

        results = []

        # Loop over all parameters
        params_name = list(self.params.keys())
        params = {
            param: (value if type(value) == list else [value])
            for param, value in self.params.items()
        }
        new_params = list(itertools.product(*params.values()))
        for param_combo in new_params:

            # Train and evaluate model
            params_to_use = {
                param: value for param, value in zip(params_name, param_combo)
            }
            output, computation_time = self._train_tm_model(params_to_use)
            scores = self.evaluate(output)

            # Update results
            result = {
                "Dataset": self.dataset,
                "Dataset Size": len(self.data.get_corpus()),
                "Model": self.model_name,
                "Params": params_to_use,
                "Scores": scores,
                "Computation Time": computation_time,
            }
            results.append(result)

        if save:
            with open(f"{save}.json", "w") as f:
                json.dump(results, f)

            try:
                from google.colab import files

                files.download(f"{save}.json")
            except ImportError:
                pass

        return results

    def _train_tm_model(
        self, params: Mapping[str, Any]
    ) -> Tuple[Mapping[str, Any], float]:
        """Select and train the Topic Model"""

        # Train BERTopic
        if self.model_name == "BERTopic":
            return self._train_bertopic(params)

    def _train_ldaseq(
        self, params: Mapping[str, any]
    ) -> Tuple[Mapping[str, Any], float]:
        """Train LDA seq model"""
        data = self.data.get_corpus()
        docs = [" ".join(words) for words in data]

        df = pd.DataFrame({"Doc": docs, "Timestamp": self.timestamps}).sort_values(
            "Timestamp"
        )
        df["Bins"] = pd.cut(df.Timestamp, bins=params["nr_bins"])
        df["Timestamp"] = df.apply(lambda row: row.Bins.left, 1)
        timestamps = df.groupby("Bins").count().Timestamp.values
        docs = df.Doc.values

        data_words = list(sent_to_words(docs))
        id2word = corpora.Dictionary(data_words)
        corpus = [id2word.doc2bow(text) for text in data_words]

        print(len(corpus), len(self.timestamps), timestamps)

        params["corpus"] = corpus
        params["id2word"] = id2word
        params["time_slice"] = timestamps
        del params["nr_bins"]

        start = time.time()
        ldaseq = ldaseqmodel.LdaSeqModel(**params)
        end = time.time()
        computation_time = end - start

        all_topics = {}
        for i in range(len(timestamps)):
            topics = ldaseq.print_topics(time=i)
            topics = [[word for word, _ in topic][:5] for topic in topics]
            all_topics[i] = {"topics": topics}

        return all_topics, computation_time


    def _train_bertopic(
        self, params: Mapping[str, any]
    ) -> Tuple[Mapping[str, Any], float]:
        """Train BERTopic model"""
        data = self.data.get_corpus()
        data = [" ".join(words) for words in data]
        params["calculate_probabilities"] = False

        if self.custom_model is not None:
            model = self.custom_model(**params)
        else:
            model = BERTopic(**params)

        start = time.time()
        topics, _ = model.fit_transform(data, self.embeddings)

        # Dynamic Topic Modeling
        if self.timestamps:
            topics_over_time = model.topics_over_time(
                data,
                topics,
                self.timestamps,
                nr_bins=self.nr_bins,
                evolution_tuning=False,
                global_tuning=False,
            )
            unique_timestamps = topics_over_time.Timestamp.unique()
            dtm_topics = {}
            for unique_timestamp in unique_timestamps:
                dtm_topic = topics_over_time.loc[
                    topics_over_time.Timestamp == unique_timestamp, :
                ].sort_values("Frequency", ascending=True)
                dtm_topic = dtm_topic.loc[dtm_topic.Topic != -1, :]
                dtm_topic = [topic.split(", ") for topic in dtm_topic.Words.values]
                dtm_topics[unique_timestamp] = {"topics": dtm_topic}

                all_words = [word for words in self.data.get_corpus() for word in words]

                updated_topics = []
                for topic in dtm_topic:
                    updated_topic = []
                    for word in topic:
                        if word not in all_words:
                            print(word)
                            updated_topic.append(all_words[0])
                        else:
                            updated_topic.append(word)
                    updated_topics.append(updated_topic)

                dtm_topics[unique_timestamp] = {"topics": updated_topics}

            output_tm = dtm_topics

        end = time.time()
        computation_time = float(end - start)

        if not self.timestamps:
            all_words = [word for words in self.data.get_corpus() for word in words]
            bertopic_topics = [
                [
                    vals[0] if vals[0] in all_words else all_words[0]
                    for vals in model.get_topic(i)[:10]
                ]
                for i in range(len(set(topics)) - 1)
            ]

            output_tm = {"topics": bertopic_topics}

        return output_tm, computation_time

    def evaluate(self, output_tm):
        """Using metrics and output of the topic model, evaluate the topic model"""
        if self.timestamps:
            results = {str(timestamp): {} for timestamp, _ in output_tm.items()}
            for timestamp, topics in output_tm.items():
                self.metrics = self.get_metrics()
                for scorers, _ in self.metrics:
                    for scorer, name in scorers:
                        score = scorer.score(topics)
                        results[str(timestamp)][name] = float(score)

        else:
            # Calculate results
            results = {}
            for scorers, _ in self.metrics:
                for scorer, name in scorers:
                    score = scorer.score(output_tm)
                    results[name] = float(score)

            # Print results
            if self.verbose:
                print("Results")
                print("============")
                for metric, score in results.items():
                    print(f"{metric}: {str(score)}")
                print(" ")

        return results

    def get_dataset(self):
        """Get dataset from OCTIS"""
        data = Dataset()

        if self.custom_dataset:
            data.load_custom_dataset_from_folder(self.dataset)
        else:
            data.fetch_dataset(self.dataset)
        return data

    def get_metrics(self):
        """Prepare evaluation measures using OCTIS"""
        npmi = Coherence(texts=self.data.get_corpus(), topk=self.topk, measure="c_npmi")
        topic_diversity = TopicDiversity(topk=self.topk)

        # Define methods
        coherence = [(npmi, "npmi")]
        diversity = [(topic_diversity, "diversity")]
        metrics = [(coherence, "Coherence"), (diversity, "Diversity")]

        return metrics


def sent_to_words(sentences):
    for sentence in sentences:
        yield (gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
topics_over_time = topic_model.topics_over_time(list(data[Description_to_process]), list(data['Day']), nr_bins=20)


6it [00:03,  1.50it/s]


In [ ]:
topics_over_time.to_csv('../drive/MyDrive/Colab Notebooks/Complaint/overtime.csv', index = False)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time)


In [ ]:
# Save the model in the previously created folder with the name 'my_best_model'
topic_model.save("../drive/MyDrive/Colab Notebooks/Complaint/my_best_model")



2024-03-11 08:41:37,115 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
# Load the serialized model
my_best_model = BERTopic.load("../drive/MyDrive/Colab Notebooks/Complaint/my_best_model")

my_best_model

In [ ]:
freq = topic_model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head()

Number of topics: 201


,Topic,Count,Name,Representation,Representative_Docs
0,-1,1621,-1_the_and_to_was,"[the, and, to, was, on, that, of, in, they, for]",[In accordance with the Fair Credit Reporting ...
1,0,328,0_debt_collection_this_company,"[debt, collection, this, company, me, was, the...",[I received a letter in the mail from a compan...
2,1,103,1_letters_continuously_thus_uploaded,"[letters, continuously, thus, uploaded, falsel...",[I'm really not sure what happened. I have mai...
3,2,88,2_car_vehicle_finance_down,"[car, vehicle, finance, down, dealership, loan...",[I have been contact with Credit Acceptance Co...
4,3,83,3_check_bank_checks_funds,"[check, bank, checks, funds, america, deposit,...",[On XX/XX/XXXX I opened an online checking and...


In [ ]:
freq.to_csv("../drive/MyDrive/Colab Notebooks/Complaint/f.csv")

In [ ]:
data=df_complaints[df_complaints['Year']==2023]
data=data[data['Month'].isin([1])]
data=data[data['Day'].isin([7])]
data.reset_index(drop=True)
text=data[Description_to_process][0]
topics, probabilites = topic_model.transform(text)

print(topics)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-03-11 08:54:04,613 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2024-03-11 08:54:09,508 - BERTopic - Dimensionality - Completed ✓
2024-03-11 08:54:09,509 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2024-03-11 08:54:09,518 - BERTopic - Cluster - Completed ✓


[24]


In [ ]:
class TopicModeling(object):
    def __init__(self,datasets, Description,penality, domain_stopwords):
        self.datasets=datasets

        self.penality=penality
        self.sentences=datasets[Description]
        self.prepocessed_words=[]

        self.number_topics=None
        self.avg_cosine_similarity=None
        self.coherence_score=None
        self.perplexity=None
        self.Best_topic_numbers=None
        self.bigram_mod=None
        self.trigram_mod=None
        self.data_words_bigrams=None
        self.data_words_trigrams=None
        self.data_lemmatized=None
        self.id2word=None
        self.corpus=None
        self.LDA_model=None
        self.topics_number=None
        self.labels=None
        self.plot=None

        self.domain_stopwords=domain_stopwords

    #run prepocessing part, and then run the LDA_Modeling part
    def Run_Prepocessing(self):
        self.text_process()
        self.build_gram_model()
        self.make_bigrams()
        self.make_trigrams()
        self.lemmatization()
        self.build_Id2Word_Corpus()

    #text process including transfer to lower case, remove punctuation, replace http to url.
    def text_process(self):
        for mess in self.sentences:
            partial_clean_word=[word.lower() for word in mess.split() if word not in stopwords.words("english")] #remove stop words
            remove_At_word=self.Words_Removed_At(partial_clean_word) #remove @people words
            new_sentence=" ".join(remove_At_word)

            #remove the punctuation
            remove_punc=[char for char in new_sentence if char not in string.punctuation] #remove punctuation
            remove_punc="".join(remove_punc)
            res_list=remove_punc.split(" ")
            res_list=[word for word in res_list if word!="" and word.startswith("http")==False] #remove http
            self.prepocessed_words.append(res_list)

    def Words_Removed_At(self,words): #remove @ people and sentences
        alist=[]
        for i in range(0,len(words)):
            word=words[i]
            if word.startswith("@")==False:
                alist.append(word)
        return alist



In [ ]:


from nltk.corpus import stopwords
import string
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
import spacy
#get_ipython().magic(u'matplotlib inline')

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


import os
import sys
import json
import pandas as pd
import numpy as np
import re
import time

class TopicModeling(object):
    def __init__(self,datasets, Description,penality, domain_stopwords):
        self.datasets=datasets

        self.penality=penality
        self.sentences=datasets[Description]
        self.prepocessed_words=[]

        self.number_topics=None
        self.avg_cosine_similarity=None
        self.coherence_score=None
        self.perplexity=None
        self.Best_topic_numbers=None
        self.bigram_mod=None
        self.trigram_mod=None
        self.data_words_bigrams=None
        self.data_words_trigrams=None
        self.data_lemmatized=None
        self.id2word=None
        self.corpus=None
        self.LDA_model=None
        self.topics_number=None
        self.labels=None
        self.plot=None

        self.domain_stopwords=domain_stopwords

    #run prepocessing part, and then run the LDA_Modeling part
    def Run_Prepocessing(self):
        self.text_process()
        self.build_gram_model()
        self.make_bigrams()
        self.make_trigrams()
        self.lemmatization()
        self.build_Id2Word_Corpus()


    #LDA topic modeling part includeing getAvgCosineSimilarity method and getTopicTenKeyWord method
    def Run_LDA_Model(self,number_topics):
        self.number_topics=number_topics
        self.build_LDA_model()
        # self.visualize_LDA_model()
        self.getAvgCosineSimilarity()
        self.getTopicTenKeyWord()
        self.updateDataset()



    #select the number of topics based on average cosine similarity, the smaller of average cosine similarity, the better the results because it can
    #seperate each topics very well.
    def select_Best_Number_Topics(self):
        best=1
        best_number_topics=0
        for i in range(0,len(self.topics_list)):
            number_topics=self.topics_list[i]
            avg_cosine_similarity=self.avg_cosine_similarity_list[i]
            if avg_cosine_similarity<best:
                best=avg_cosine_similarity
                best_number_topics=number_topics
        self.Best_topic_numbers=best_number_topics


    #make the plot of average cosine similarity.
    def MakePlot(self):
        X=self.topics_list
        Y=self.avg_cosine_similarity_list

        plt.plot(X,Y)
        plt.xlabel("The number of clusters")
        plt.ylabel("The Avg cosine similarity score")
        plt.label("The number of culsters with the Avg cosine similarity score list")
        plt.show()


    #print out the topic word
    def printOutTopicWords(self):
        result=TopicModeling(self.dataset,self.Best_topic_numbers,self.penality)
        result.Run_LDA_Model()   #result is the class
        final_model=result.LDA_model
        plot=result.visualize_LDA_model()
        #return plot



    #text process including transfer to lower case, remove punctuation, replace http to url.
    def text_process(self):
        for mess in self.sentences:
            partial_clean_word=[word.lower() for word in mess.split() if word not in stopwords.words("english")] #remove stop words
            remove_At_word=self.Words_Removed_At(partial_clean_word) #remove @people words
            new_sentence=" ".join(remove_At_word)

            #remove the punctuation
            remove_punc=[char for char in new_sentence if char not in string.punctuation] #remove punctuation
            remove_punc="".join(remove_punc)
            res_list=remove_punc.split(" ")
            res_list=[word for word in res_list if word!="" and word.startswith("http")==False] #remove http
            self.prepocessed_words.append(res_list)

    def Words_Removed_At(self,words): #remove @ people and sentences
        alist=[]
        for i in range(0,len(words)):
            word=words[i]
            if word.startswith("@")==False:
                alist.append(word)
        return alist


    #build the gram model
    def build_gram_model(self):
        bigram = gensim.models.Phrases(self.prepocessed_words, min_count=5, threshold=100) # higher threshold fewer phrases.
        trigram = gensim.models.Phrases(bigram[self.prepocessed_words], threshold=100)

        # Faster way to get a sentence clubbed as a trigram/bigram
        bigram_mod = gensim.models.phrases.Phraser(bigram)
        trigram_mod = gensim.models.phrases.Phraser(trigram)

        self.bigram_mod=bigram_mod
        self.trigram_mod=trigram_mod

    def make_bigrams(self):
        self.data_words_bigrams=[self.bigram_mod[doc] for doc in self.prepocessed_words]

    def make_trigrams(self):
        self.data_words_trigrams=[self.trigram_mod[self.bigram_mod[doc]] for doc in self.prepocessed_words]


    #words lemmatization. only allow the non, adj, verb, adv words. it will remove other types of words in a sentence.
    def lemmatization(self,allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        nlp = spacy.load('en', disable=['parser', 'ner'])
        texts_out = []
        for sent in self.data_words_bigrams:
            doc = nlp(" ".join(sent))
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        self.data_lemmatized=texts_out


    #build the id2word corpus
    def build_Id2Word_Corpus(self):
        self.id2word=corpora.Dictionary(self.data_lemmatized)
        self.corpus=[self.id2word.doc2bow(text) for text in self.data_lemmatized]


    #build the LDA topic modeling by using some empirical parameters.
    def build_LDA_model(self):
        self.LDA_model=gensim.models.ldamodel.LdaModel(corpus=self.corpus,
                                           id2word=self.id2word,
                                           num_topics=self.number_topics,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto",
                                           per_word_topics=True)

    def getLDA_model(self):
        return self.LDA_model


    #visualize the LDA model by the power of pyLDAvis package
    def visualize_LDA_model(self):
        #pyLDAvis.enable_notebook()
        vis=pyLDAvis.gensim.prepare(self.LDA_model,self.corpus,self.id2word)
        self.plot=vis

        #return vis

    def getId2Word(self):
        return self.id2word

    def getCorpus(self):
        return self.corpus



    #get the average cosine similarity
    def getAvgCosineSimilarity(self):
        Sum=0
        topics=self.LDA_model.print_topics()
        topics_number=len(topics)
        for j in range(0,len(topics)-1):
            sentence_1=self.getTopicScoreList(topics[j][1])
            for k in range(j+1,len(topics)):
                sentence_2=self.getTopicScoreList(topics[k][1])
                Sum+=self.getCosineSimilarity(sentence_1,sentence_2)
        avg_cosine=Sum/(topics_number*(topics_number-1)/2)
        avg_cosine+=(self.penality*topics_number)
        return avg_cosine

    #get the perplexity of current model
    def getPerplexity(self):
        perplexity=self.LDA_model.log_perplexity(self.corpus)
        return perplexity

    #get the coherence score of the current model
    def getCoherence(self):
        coherence_model_lda = CoherenceModel(model=self.LDA_model,texts=self.data_lemmatized, dictionary=self.id2word, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        return coherence_lda



    #helper function to get Cosine similarity
    def getTopicScoreList(self,sentence):
        adict=dict()
        for atuple in sentence.split("+"):
            tuple_list=[]
            word=atuple.split("*")[1]
            number=float(atuple.split("*")[0])
            adict[word]=number
        return adict


    #get the cosine similarity of two sentences
    def getCosineSimilarity(self,sentence1,sentence2):
        Sum=0
        for key1 in sentence1:
            if key1 in sentence2:
                Sum+=sentence1[key1]*sentence2[key1]

        Sum_1=0
        for key1 in sentence1:
            Sum_1+=sentence1[key1]**2
        Sum_1=np.sqrt(Sum_1)

        Sum_2=0
        for key2 in sentence2:
            Sum_2+=sentence2[key2]**2
        Sum_2=np.sqrt(Sum_2)

        return Sum/(Sum_1*Sum_2)

    #get the top 10 key word
    def getTopicTenKeyWord(self):
        sentences=self.LDA_model.print_topics()
        all_topic=[]
        for i in range(0,len(sentences)):
            onesentence=sentences[i][1] #all string
            number=0
            one_topic=[]
            for j in range(0,len(onesentence.split("+"))):
                number_word=onesentence.split("+")[j]   #the format is like "0.082*"nepal" '
                mess_word=number_word.split("*")[1]
                clean_word=self.getCleanWord(mess_word)
                if number<10:
                    one_topic.append(clean_word)
                    number+=1
            one_topic=" ".join(one_topic)
            all_topic.append(one_topic)

        #then we will print out every key word in the topic list

        self.topic_names=all_topic

    def getEachTopicNames(self):
        for m in range(0,len(self.topic_names)):
            print("The "+str(m+1)+" Topics Meanings are: "+str(self.topic_names[m]))


    def getCleanWord(self,aword):
        new_word=""
        for letter in aword:
            if letter!="'" and letter!='"' and letter!=" ":
                new_word+=letter
        return new_word
        new_word=new_word.strip()



    def getPercentageAndTopics(self):
        percent_Topic =self.LDA_model[self.corpus]
        percent_list=[]
        topic_list=[]

        for i in range(0,len(percent_Topic)):
            res=percent_Topic[i][0]

            highest_percent=0
            best_topic=None
            for j in range(0,len(res)):
                if res[j][1]>highest_percent:
                    highest_percent=res[j][1]
                    best_topic=self.topic_names[j]

            percent_list.append(highest_percent)
            topic_list.append(best_topic)
        return percent_list,topic_list


    def updateDataset(self):
        percent_list,topic_list=self.getPercentageAndTopics()
        self.datasets["Topic_Percent"]=percent_list
        self.datasets["Topic"]=topic_list

